In [15]:
import glob
import pandas as pd
import numpy as np

In [16]:
appended_data = []
for f in glob.glob ("Data - NSW/*.csv"):
    df = pd.read_csv(f, header=0)
    appended_data.append(df)
df = pd.concat(appended_data)
df.to_csv('appended.csv')

In [17]:
print(df.dtypes)

REGION             object
SETTLEMENTDATE     object
TOTALDEMAND       float64
RRP               float64
PERIODTYPE         object
dtype: object


In [18]:
df['Date Time'] = pd.to_datetime(df['SETTLEMENTDATE'])
df['Day'] = df["Date Time"].dt.date
df['Month'] = df['Date Time'].dt.month
df['Year'] = df['Date Time'].dt.year
df['Hour'] = df['Date Time'].dt.hour
df['Minute'] = df['Date Time'].dt.minute

df['Demand'] = pd.to_numeric(df['TOTALDEMAND'], errors='coerce')
print(df.head())

  REGION       SETTLEMENTDATE  TOTALDEMAND    RRP PERIODTYPE  \
0   NSW1  2021/01/01 00:30:00      6846.57  38.04      TRADE   
1   NSW1  2021/01/01 01:00:00      6610.45  38.10      TRADE   
2   NSW1  2021/01/01 01:30:00      6268.29  32.26      TRADE   
3   NSW1  2021/01/01 02:00:00      6043.20  32.45      TRADE   
4   NSW1  2021/01/01 02:30:00      5826.44  27.76      TRADE   

            Date Time         Day  Month  Year  Hour  Minute   Demand  
0 2021-01-01 00:30:00  2021-01-01      1  2021     0      30  6846.57  
1 2021-01-01 01:00:00  2021-01-01      1  2021     1       0  6610.45  
2 2021-01-01 01:30:00  2021-01-01      1  2021     1      30  6268.29  
3 2021-01-01 02:00:00  2021-01-01      1  2021     2       0  6043.20  
4 2021-01-01 02:30:00  2021-01-01      1  2021     2      30  5826.44  


In [19]:
print(df.dtypes)

REGION                    object
SETTLEMENTDATE            object
TOTALDEMAND              float64
RRP                      float64
PERIODTYPE                object
Date Time         datetime64[ns]
Day                       object
Month                      int64
Year                       int64
Hour                       int64
Minute                     int64
Demand                   float64
dtype: object


In [20]:
df.drop(['REGION'], axis=1, inplace=True)
df.drop(['SETTLEMENTDATE'], axis=1, inplace=True)
df.drop(['TOTALDEMAND'], axis=1, inplace=True)
df.drop(['RRP'], axis=1, inplace=True)
df.drop(['PERIODTYPE'], axis=1, inplace=True)
df.drop(['Date Time'], axis=1, inplace=True)

In [21]:
df.head()

,Day,Month,Year,Hour,Minute,Demand
0,2021-01-01,1,2021,0,30,6846.57
1,2021-01-01,1,2021,1,0,6610.45
2,2021-01-01,1,2021,1,30,6268.29
3,2021-01-01,1,2021,2,0,6043.20
4,2021-01-01,1,2021,2,30,5826.44


In [27]:
X = []
y = []
for i in range (0, df.shape[0]-48):
    X.append(df.iloc[i:i+48, 5])
    y.append(df.iloc[i+48, 5])

X, y = np.array(X), np.array(y)
y = np.reshape(y, (len(y), 1))

X = np.delete(X, list(range(1, X.shape[1], 2)), axis=1)
X = np.delete(X, list(range(1, X.shape[0], 2)), axis=0)
y = np.delete(y, list(range(1, y.shape[0], 2)), axis=0)

pd.DataFrame(X).to_csv('appended_DemandX.csv')
pd.DataFrame(y).to_csv('appended_Demandy.csv')

In [29]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))

X = scaler.fit_transform(X)
y = scaler.fit_transform(y)

X_train, X_test = X[:-480], X[-480:]
y_train, y_test = y[:-480], y[-480:]

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [34]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, LSTM
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [32]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(128, return_sequences=False))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

In [ ]:
import os
import tensorflow as tf
tf.debugging.set_log_device_placement(True)
epochs=1000
logdir='logs1'
checkpoint_path = 'checkpoint/cp1.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
cb_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path,
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=True
)

In [41]:
filepath = 'model/lstm1.hdf5'
callbacks = [EarlyStopping(monitor='val_loss', patience=50),tensorboard_callback, ModelCheckpoint(filepath, monitor='val_loss',verbose=1, save_best_only=True, mode='min',save_weights_only=True)]
optimizers.Adam(learning_rate=1e-4)

In [43]:
model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mape'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 24, 50)            10400     
                                                                 
 lstm_1 (LSTM)               (None, 50)                20200     
                                                                 
 dense (Dense)               (None, 50)                2550      
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 33,201
Trainable params: 33,201
Non-trainable params: 0
_________________________________________________________________


In [44]:
history = model.fit(X_train, y_train, validation_split=0.2, epochs=1000, callbacks=callbacks, batch_size=8)

Epoch 1/1000
4983/4983 [==============================] - 78s 15ms/step - loss: 1.5687e-04 - mae: 0.0091 - mape: 400.3061 - val_loss: 1.9847e-04 - val_mae: 0.0110 - val_mape: 2.1944
Epoch 2/1000
4983/4983 [==============================] - 70s 14ms/step - loss: 1.5079e-04 - mae: 0.0089 - mape: 518.1671 - val_loss: 1.3052e-04 - val_mae: 0.0086 - val_mape: 1.6623
Epoch 3/1000
4983/4983 [==============================] - 65s 13ms/step - loss: 1.4588e-04 - mae: 0.0088 - mape: 529.1026 - val_loss: 2.0689e-04 - val_mae: 0.0107 - val_mape: 1.9749
Epoch 4/1000
4983/4983 [==============================] - 66s 13ms/step - loss: 1.4257e-04 - mae: 0.0087 - mape: 254.3635 - val_loss: 2.8353e-04 - val_mae: 0.0138 - val_mape: 2.6575
Epoch 5/1000
4983/4983 [==============================] - 66s 13ms/step - loss: 1.3499e-04 - mae: 0.0085 - mape: 46.2049 - val_loss: 1.6118e-04 - val_mae: 0.0096 - val_mape: 1.8587
Epoch 6/1000
4983/4983 [==============================] - 65s 13ms/step - loss: 1.3647e-04 

In [ ]:
MSE, MAE = model.evaluate(X_test, y_test)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.epoch, history.history["loss"], 'g', label='Training loss')
plt.plot(history.epoch, history.history["val_loss"], 'b', label='validation loss')

plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig('MSElstm.png', bbox_inches='tight')
plt.show()

In [ ]:
plt.plot(history.epoch, history.history["mae"], 'g', label='Training accuracy')
plt.plot(history.epoch, history.history["val_mae"], 'b', label='Validation accuracy')

plt.title('MAE')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()
plt.savefig('MAE.png', bbox_inches='tight')
plt.show()

In [ ]:
plt.plot(history.epoch, history.history["mape"], 'g', label='Training accuracy')
plt.plot(history.epoch, history.history["val_mape"], 'b', label='Validation accuracy')

plt.title('MAPE')
plt.xlabel('Epochs')
plt.ylabel('MAPE')
plt.legend()
plt.savefig('MAPE.png', bbox_inches='tight')
plt.show()

In [ ]:
import time

for i in range (0, X_test.shape[0]):
    demand_summary = []
    X_input = X_test[i, :, :]
    X_input = np.reshape(X_input, (1, X_input.shape[0], 1))
    X_input = model.predict(X_input)
    forecast = scaler.inverse_transform(X_input)

    y_input = y_test[i,:]
    y_input = np.reshape(y_input, (1,1))
    actual = scaler.inverse_transform(actual)
    

    demand_summary.append(actual)
    demand_summary.extend(forecast)

    df_animate = pd.DataFrame(demand_summary)
    df_animate = df_animate.T
    df_animate.to_csv('real_time_demand.csv', mode='a', header=False, index=False)

    print(demand_summary)
    time.sleep(0.5)